## Assignment week 04: "North Navarra buissnes startup" . By CPC

#### 1) Importing Lib

In [5]:
# Importing Libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
from urllib.request import urlopen # library to open the url from which we would like to extract the data.
#from bs4 import BeautifulSoup  # get the HTML structure of the page that we want to work with.
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import k-means from clustering stage
from sklearn.cluster import KMeans


pd.__version__

'0.25.0'

#### 2) Extracting Data base form wikipedia tables and creating a Data Frame with Coordinates

In [ ]:
NorteNavarra=pd.read_html('https://es.wikipedia.org/wiki/Merindad_de_Pamplona')
df=NorteNavarra[2]
df = df.drop(['Nombre'], axis = 1)
df.rename(columns = {'Nombre oficial' : 'Village_Name', 'Municipio' : 'Municip', 'Categoría' : 'Category','Población 2014[3]​':'Population'  } , inplace = True)
df['Latitude']=None
df['Longitude']=None 
print(df.shape)
print(df.columns)
df.head()

In [ ]:
#Adding coordinates
j = 0
for i in df['Village_Name']:
    address = i+", Navarre, Spain"
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    if location:
        df.iloc[j,4] = location.latitude
        df.iloc[j,5] = location.longitude
        j=j+1
    else:
        j=j+1

In [ ]:
#Filter the database from Null values
#df.isnull().sum()   ##372
df=df.dropna()
df.shape
df.head()

In [ ]:
latmap = df.Latitude.mean()
lonmap = df.Longitude.mean()

#### 3) Visualizing Maps with Folium

In [ ]:
#importing Maps
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium installed and imported!')

In [ ]:
#Creating Navarra's map
navarra_map = folium.Map(location=[latmap, lonmap], zoom_start=10)    

In [ ]:
# add markers to map
for lat, lng, village, population in zip(df['Latitude'], df['Longitude'], df['Village_Name'], df['Population']):
    label = '{}, {}'.format(village, population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(navarra_map)  
    
navarra_map

#### 4) Downloading venues Data from Foursquare

#### Extracting Data 

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
LIMIT = 100
latitude1= df.loc[2, 'Latitude']
longitude1=df.loc[2, 'Longitude']
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)
url
# &limit={} , LIMIT    &intent=browse


In [ ]:
def obtenervenues(names, latitudes, longitudes, radius=2000, LIMIT=100):
    
    venues_list=pd.DataFrame({'nombre':[],'location.distance':[],'name':[],'categories':[], 'location.lat':[], 'location.lng':[],'id':[]})
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        results = requests.get(url).json()["response"]['venues']
        
        #tranform venues into a dataframe
        dataframe = json_normalize(results)
        dataframe=dataframe[dataframe['location.distance']<radius]
        #filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
        #dataframe_filtered = dataframe.loc[:, filtered_columns]
        dataframe['nombre']=name
        dataframe=dataframe[['nombre','location.distance','name','categories', 'location.lat', 'location.lng','id']]
        
        venues_list=venues_list.append(dataframe, sort=False)
      
    
    return(venues_list)


In [ ]:
navarra_venues = obtenervenues(names=df['Village_Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
#navarra_venues.to_csv('navarra_venuesfull.csv')

In [7]:
print('There are {} uniques categories.'.format(len(navarra_venues['categories'].unique())))
print('There are {} uniques Village.'.format(len(navarra_venues['nombre'].unique())))


There are 308 uniques categories.
There are 316 uniques Village.


In [29]:
# Based on costumer surveys (154 categories). The most atracctive category Venues in the list. first We need to fit data 

navarra_venues.categories.replace("Dentist's Office", "Dentist", inplace = True)
navarra_venues.categories.replace("Men's Store", "Men Store", inplace = True)
navarra_venues.categories.replace("Doctor's Office", "Doctors Office", inplace = True)

category_costumer=  {'cat': ['Gas Station','Office','Field','Other Great Outdoors','Bank','Factory','General Entertainment','Mountain','Plaza','Bakery','Post Office','Playground','Grocery Store','Pharmacy','Pool','Bus Line','Salon / Barbershop','Trail','Scenic Lookout','Medical Center','Butcher','Park','Outdoors & Recreation','Food','Doctors Office','Supermarket','School','Campground','Gym / Fitness Center','General Travel','Hardware Store','Soccer Field','Farm','Museum','College Library','Professional & Other Places','Residential Building (Apartment / Condo)','Theme Park Ride / Attraction','Food & Drink Shop','Department Store','Brewery','Electronics Store','Public Art','Lake','Athletics & Sports','History Museum','Student Center','Concert Hall','Government Building','Library','National Park','Hospital','Farmers Market','Garden','Pub','Bookstore','Police Station','College Classroom','Dentists Office','Sporting Goods Shop','Laundry Service','Bike Shop','Mobile Phone Shop','Gym Pool','Forest','Elementary School','Community College','Basketball Court','Rock Club','River','Performing Arts Venue','Theme Park','College Soccer Field','College Bookstore','Rock Climbing Spot','Skate Park','Stadium','Train Station','Convenience Store','Internet Cafe','College Academic Building','Beer Garden','Outdoor Sculpture','Radio Station','Theater','Health Food Store','Warehouse','Veterinarian','Beach','Bus Stop','Beer Bar','Business Center','College Gym','Recreation Center','Tennis Stadium','Skating Rink','Social Club','Community Center','Airport Gate','Airport','Movie Theater','General College & University','Arts & Entertainment','Volcano','Baseball Field','Art Museum','Bike Trail','Bus Station','Fire Station','Middle School','Island','Spa','Recycling Facility','Soccer Stadium','College Administrative Building','College Communications Building','Convention Center','Historic Site','Music Venue','Building','Miscellaneous Shop','Housing Development','Coworking Space','Shopping Mall','Clothing Store','Bridge','Gym','Rest Area','Shoe Store','EV Charging Station','Assisted Living','Accessories Store','Men Store','Gift Shop','Food Court','Tech Startup','High School','Arts & Crafts Store','Auditorium','Market','Capitol Building','Shop & Service','Conference Room','Stationery Store','Indie Movie Theater','Paper / Office Supplies Store','Travel & Transport','Art Gallery','Town Hall','Health & Beauty Service','Financial or Legal Service','Medical Lab','Festival','Campaign Office',
]}
filter = pd.DataFrame(category_costumer, columns= ['cat'])
navarra_venues['filter']=navarra_venues.categories.isin(filter.cat)
navarra_venues=navarra_venues[navarra_venues['filter']==True]
navarra_venues = navarra_venues.drop(['filter'], axis = 1)

In [72]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
navarra_grouped = navarra_onehot.groupby('nombre').mean().reset_index()
navarra_grouped.shape

(276, 154)

In [73]:
# Print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in navarra_grouped['nombre']:
    print("----"+hood+"----")
    temp = navarra_grouped[navarra_grouped['nombre'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adiós----
              venue  freq
0  Capitol Building  0.33
1              Park  0.33
2             Field  0.33
3       Post Office  0.00
4              Pool  0.00


----Adériz----
                   venue  freq
0             Campground  0.25
1  Outdoors & Recreation  0.25
2               Mountain  0.25
3                  Field  0.25
4                   Pool  0.00


----Aitzano----
                  venue  freq
0                 Field  0.29
1              Building  0.14
2   Housing Development  0.14
3                  Pool  0.14
4  Gym / Fitness Center  0.14


----Aizarotz----
               venue  freq
0           Building   1.0
1  Accessories Store   0.0
2               Park   0.0
3      National Park   0.0
4             Office   0.0


----Aizkorbe----
                  venue  freq
0                Bakery   0.2
1           Post Office   0.2
2  Gym / Fitness Center   0.2
3                 Trail   0.2
4        Scenic Lookout   0.2


----Aizoáin----
                   venue  freq


In [74]:
# Let's put that into a *pandas* dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['nombre']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['nombre'] = navarra_grouped['nombre']

for ind in np.arange(navarra_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(navarra_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

In [96]:
neighborhoods_venues_sorted.head()

,nombre,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adiós,Field,Capitol Building,Park,Community College,Concert Hall,Conference Room,Convenience Store,Convention Center,Community Center,Festival
1,Adériz,Outdoors & Recreation,Field,Campground,Mountain,Warehouse,Farm,Factory,Elementary School,Electronics Store,EV Charging Station
2,Aitzano,Field,Pool,Housing Development,Scenic Lookout,Building,Gym / Fitness Center,Community College,Concert Hall,Community Center,Conference Room
3,Aizarotz,Building,Warehouse,Doctors Office,Farmers Market,Farm,Factory,Elementary School,Electronics Store,EV Charging Station,Department Store
4,Aizkorbe,Post Office,Bakery,Gym / Fitness Center,Trail,Scenic Lookout,Factory,Elementary School,Electronics Store,EV Charging Station,Doctors Office


#### 5) Clustering DATA, K=5

In [103]:
#Run *k*-means to cluster the neighborhood into 5 clusters.
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

navarra_grouped_clustering = navarra_grouped.drop('nombre', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(navarra_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [104]:
#navarra_grouped_clustering.head()
neighborhoods_venues_sorted.groupby('Cluster Labels').count().sort_values(['Cluster Labels'], ascending=False)

,nombre,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,
4,32,32,32,32,32,32,32,32,32,32,32
3,11,11,11,11,11,11,11,11,11,11,11
2,13,13,13,13,13,13,13,13,13,13,13
1,21,21,21,21,21,21,21,21,21,21,21
0,199,199,199,199,199,199,199,199,199,199,199


In [16]:
#Run *k*-means to cluster the neighborhood into 5 clusters.
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

navarra_grouped_clustering = navarra_grouped.drop('nombre', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(navarra_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [17]:
#navarra_grouped_clustering.head()
neighborhoods_venues_sorted.groupby('Cluster Labels').count().sort_values(['Cluster Labels'], ascending=False)

,nombre,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,
4,10,10,10,10,10,10,10,10,10,10,10
3,12,12,12,12,12,12,12,12,12,12,12
2,243,243,243,243,243,243,243,243,243,243,243
1,35,35,35,35,35,35,35,35,35,35,35
0,16,16,16,16,16,16,16,16,16,16,16


In [18]:
navarra_merged = df
navarra_merged = navarra_merged.join(neighborhoods_venues_sorted.set_index('nombre'), on='Village_Name')
navarra_merged=navarra_merged[navarra_merged['Cluster Labels']<5]
navarra_merged['Cluster Labels'] = navarra_merged['Cluster Labels'].astype(int)

In [23]:
navarra_merged[10:16]

,Village_Name,Municip,Category,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Aldatz,Larráun,Concejo,130,43.009755,-1.859486,1,Restaurant,Field,Beach,Yoga Studio,Driving School,Dry Cleaner,EV Charging Station,Electronics Store,Elementary School,Event Space
13,Alkaiaga,Lesaca,Barrio,201,43.279708,-1.694239,2,Bar,Diner,Spanish Restaurant,Hotel Bar,Building,Gas Station,Restaurant,Café,Mountain,Factory
14,Alkainzuriain,Goizueta,Barrio,24,43.160850,-1.874923,2,Cemetery,Gas Station,Yoga Studio,Dog Run,Driving School,Dry Cleaner,EV Charging Station,Electronics Store,Elementary School,Event Space
15,Almandoz,Baztán,Lugar,198,43.090510,-1.605399,4,Mountain,Food,Farm,Dry Cleaner,EV Charging Station,Electronics Store,Elementary School,Event Space,Factory,Falafel Restaurant
16,Alkerdi,Urdax,Barrio,85,43.276305,-1.528310,2,Restaurant,Spanish Restaurant,Bar,History Museum,Food,BBQ Joint,Mountain,General Entertainment,Café,Scenic Lookout
17,Altsasu/Alsasua,Alsasua,Villa,7612,42.894966,-2.168510,2,Bar,Building,Office,Restaurant,Electronics Store,Bank,Train Station,Gas Station,Dessert Shop,Spanish Restaurant


#### 6) Mapping clusters and seclection of best places according to the company interest

In [25]:
#Creating Navarra's map
latmap = df.Latitude.mean()
lonmap = df.Longitude.mean()
navarra_map = folium.Map(location=[latmap, lonmap], zoom_start=10)    

In [26]:
# create map
map_clusters = folium.Map(location=[latmap, lonmap], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(navarra_merged['Latitude'], navarra_merged['Longitude'], navarra_merged['Village_Name'], navarra_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters